In [1]:
# Import the labeled Data
import pandas as pd
df = pd.read_csv("orders_labeled.csv")

In [2]:
# Preview the data
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,payment_type,payment_installments,payment_value,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,churn
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,credit_card,1.0,18.12,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,1
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,voucher,1.0,2.00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,1
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,voucher,1.0,18.59,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,1
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,boleto,1.0,141.46,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,0
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,credit_card,3.0,179.12,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,0


In [3]:
# Total number of orders per customer # Feature 1
orders_per_customer = df.groupby("customer_unique_id")["order_id"].nunique().reset_index()
orders_per_customer.columns = ["customer_unique_id", "num_orders"]

In [4]:
# Total money spent by each customer # Feature 2
payment_per_customer = df.groupby("customer_unique_id")["payment_value"].sum().reset_index()
payment_per_customer.columns = ["customer_unique_id", "total_payment"]

In [5]:
# Average payment per order # Feature 3
avg_payment = df.groupby("customer_unique_id")["payment_value"].mean().reset_index()
avg_payment.columns = ["customer_unique_id", "avg_payment"]

In [6]:
# Days since last purchase # feature 4
## Convert timestamp
df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"])

## Last purchase date
last_purchase_date = df.groupby("customer_unique_id")["order_purchase_timestamp"].max().reset_index()
last_purchase_date.columns = ["customer_unique_id", "last_purchase_date"]

## Reference date = latest date in the dataset
reference_date = df["order_purchase_timestamp"].max()

## Days since last purchase
last_purchase_date["days_since_last_purchase"] = (reference_date - last_purchase_date["last_purchase_date"]).dt.days

In [7]:
# Merge all features into one table
features = orders_per_customer.merge(payment_per_customer, on="customer_unique_id")
features = features.merge(avg_payment, on="customer_unique_id")
features = features.merge(last_purchase_date[["customer_unique_id", "days_since_last_purchase"]], on="customer_unique_id")
features = features.merge(df[["customer_unique_id", "churn"]].drop_duplicates(), on="customer_unique_id")

In [8]:
# Convert it into a CVS File
features.to_csv("churn_features.csv", index=False)

In [9]:
# Preview the result
features.head()

,customer_unique_id,num_orders,total_payment,avg_payment,days_since_last_purchase,churn
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.90,141.90,115,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,27.19,118,0
2,0000f46a3911fa3c0805444483337064,1,86.22,86.22,541,1
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.62,43.62,325,1
4,0004aac84e0df4da2b147fca70cf8255,1,196.89,196.89,292,1
